In [3]:
import os
import pandas as pd
import numpy as np
import datetime
import pickle
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", 50)

target_folder_path = '/Volumes/Database/TEJ_rawdata'
%cd "{target_folder_path}"

/Volumes/Database/TEJ_rawdata


In [7]:
price_data = pd.read_table(rf"{target_folder_path}/adj.txt", encoding='Big5-HKSCS', low_memory=False)
price_data

,證券代碼,簡稱,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),報酬率％,週轉率％,流通在外股數(千股),市值(百萬元),漲跌停
0,1101,台泥,19800104,0.4446,0.4553,0.4446,0.4535,259,7000,2.0080,0.0979,264500,6718,
1,1101,台泥,19800105,0.4571,0.4606,0.4535,0.4588,270,7000,1.1811,0.1021,264500,6797,
2,1101,台泥,19800107,0.4588,0.4713,0.4588,0.4642,312,8000,1.1673,0.1180,264500,6877,
3,1101,台泥,19800108,0.4642,0.4696,0.4606,0.4606,224,6000,-0.7692,0.0847,264500,6824,
4,1101,台泥,19800109,0.4624,0.4660,0.4588,0.4642,159,4000,0.7752,0.0601,264500,6877,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9415102,8934,衡平,20201111,34.0000,34.0000,34.0000,34.0000,7,242,3.0303,0.0823,8511,289,
9415103,8934,衡平,20201112,34.5000,34.5000,34.1000,34.1000,62,2135,0.2941,0.7286,8511,290,
9415104,8934,衡平,20201113,34.5000,36.5000,34.3000,36.5000,94,3286,7.0381,1.1046,8511,311,
9415105,8934,衡平,20201116,36.0000,36.0000,36.0000,36.0000,30,1108,-1.3699,0.3525,8511,306,


In [8]:
columns_list = price_data.columns.tolist()

### make all_columns to str
for column in columns_list:
    price_data[column] = price_data[column].astype(str).str.strip()

ohlcv_list = price_data.columns.tolist()[3:-1]
ohlcv_list

['開盤價(元)',
 '最高價(元)',
 '最低價(元)',
 '收盤價(元)',
 '成交量(千股)',
 '成交值(千元)',
 '報酬率％',
 '週轉率％',
 '流通在外股數(千股)',
 '市值(百萬元)']

In [12]:
price_data['年月日'] = pd.to_datetime(price_data['年月日'], format='%Y%m%d')
# data_full['漲跌停'] =  data_full['漲跌停'].astype(str).str.strip()
# data_full['證券代碼'] = data_full['證券代碼'].str.extract(r'^(\d+)')
string_list = ['證券代碼']
for column_name in string_list:
    price_data[column_name] = price_data[column_name]

numeric_list = ohlcv_list
for column_name in numeric_list:
    price_data[column_name] = pd.to_numeric(price_data[column_name], errors='coerce')


price_data.rename(columns ={'證券代碼':'symbol', 
                            '年月日':'datetime', 
                            '開盤價(元)': 'adj_open',
                            '最高價(元)': 'adj_high',
                            '最低價(元)': 'adj_low', 
                            '收盤價(元)': 'adj_close',
                            '成交量(千股)': 'volume',
                            '成交值(千元)': 'volvalue',
                            '報酬率％': 'returns',
                            '週轉率％': 'turnover',
                            '流通在外股數(千股)': 'outstanding',
                            '市值(百萬元)': 'cap',
                            '漲跌停': 'limit'}, inplace=True)


In [13]:
ohlcv_list_english = price_data.columns.tolist()[3:]
ohlcv_list_english

['adj_open',
 'adj_high',
 'adj_low',
 'adj_close',
 'volume',
 'volvalue',
 'returns',
 'turnover',
 'outstanding',
 'cap',
 'limit']

In [14]:
for column in ohlcv_list_english:
    unstack_ohlcv = price_data.reset_index().set_index(['datetime','symbol'])[column].unstack()
    csv_file = os.path.join(f'{target_folder_path}/factor_data', f'{column}.csv')
    unstack_ohlcv.to_csv(csv_file)

### Get ohlcv_df

In [4]:
price_data = pd.read_table(rf"{target_folder_path}/adj.txt", encoding='Big5-HKSCS', low_memory=False)
price_data

,證券代碼,簡稱,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),報酬率％,週轉率％,流通在外股數(千股),市值(百萬元),漲跌停
0,1101,台泥,19800104,0.4446,0.4553,0.4446,0.4535,259,7000,2.0080,0.0979,264500,6718,
1,1101,台泥,19800105,0.4571,0.4606,0.4535,0.4588,270,7000,1.1811,0.1021,264500,6797,
2,1101,台泥,19800107,0.4588,0.4713,0.4588,0.4642,312,8000,1.1673,0.1180,264500,6877,
3,1101,台泥,19800108,0.4642,0.4696,0.4606,0.4606,224,6000,-0.7692,0.0847,264500,6824,
4,1101,台泥,19800109,0.4624,0.4660,0.4588,0.4642,159,4000,0.7752,0.0601,264500,6877,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9415102,8934,衡平,20201111,34.0000,34.0000,34.0000,34.0000,7,242,3.0303,0.0823,8511,289,
9415103,8934,衡平,20201112,34.5000,34.5000,34.1000,34.1000,62,2135,0.2941,0.7286,8511,290,
9415104,8934,衡平,20201113,34.5000,36.5000,34.3000,36.5000,94,3286,7.0381,1.1046,8511,311,
9415105,8934,衡平,20201116,36.0000,36.0000,36.0000,36.0000,30,1108,-1.3699,0.3525,8511,306,


In [5]:
columns_list = price_data.columns.tolist()

### make all_columns to str
for column in columns_list:
    price_data[column] = price_data[column].astype(str).str.strip()

ohlcv_list = price_data.columns.tolist()[3:]
ohlcv_list

['開盤價(元)',
 '最高價(元)',
 '最低價(元)',
 '收盤價(元)',
 '成交量(千股)',
 '成交值(千元)',
 '報酬率％',
 '週轉率％',
 '流通在外股數(千股)',
 '市值(百萬元)',
 '漲跌停']

In [5]:
price_data['年月日'] = pd.to_datetime(price_data['年月日'], format='%Y%m%d')
# data_full['漲跌停'] =  data_full['漲跌停'].astype(str).str.strip()
# data_full['證券代碼'] = data_full['證券代碼'].str.extract(r'^(\d+)')
string_list = ['證券代碼']
for column_name in string_list:
    price_data[column_name] = price_data[column_name]

numeric_list = ohlcv_list
for column_name in numeric_list:
    price_data[column_name] = pd.to_numeric(price_data[column_name], errors='coerce')


price_data.rename(columns ={'證券代碼':'symbol', 
                            '年月日':'datetime', 
                            '開盤價(元)': 'open',
                            '最高價(元)': 'high',
                            '最低價(元)': 'low', 
                            '收盤價(元)': 'close',
                            '成交量(千股)': 'volume',
                            '報酬率％': 'return',
                           '成交量(千股)':'volume'}, inplace=True)
price_data

ValueError: time data "1980-01-04" doesn't match format "%Y%m%d", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [6]:
unstack_ohlcv = price_data.reset_index().set_index(['datetime','symbol'])['開盤價(元)'].unstack()
unstack_ohlcv

KeyError: "None of ['datetime', 'symbol'] are in the columns"

In [5]:
ohlcv_list_english = price_data.columns.tolist()[3:]
ohlcv_list_english

['open', 'high', 'low', 'close', 'volume', 'return']

In [6]:
for column in ohlcv_list_english:
    unstack_ohlcv = price_data.reset_index().set_index(['datetime','symbol'])[column].unstack()
    csv_file = os.path.join('/Volumes/Database/Factor_data', f'{column}.csv')
    unstack_ohlcv.to_csv(csv_file)
